# Trabalho de Algoritmos para Big Data - 2022/2023

## Utilização de sistemas de bike-sharing - Londres

#### Formulação de problemas: 

1. Para dada temperatura e dia da semana, prever a afluencia de rentals para cada estação

In [1]:
%%bash
pwd

/


Criar uma SparkSession

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession\
    .builder\
    .appName("BigData_TG")\
    .config("spark.sql.shuffle.partitions",6)\
    .config("spark.sql.repl.eagereval.enabled",True)\
    .getOrCreate()


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1685053088962_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Some general imports 

import os
import sys

import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# from ydata_profiling import ProfileReport
import warnings
warnings.filterwarnings("ignore")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
No module named 'pandas'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'pandas'



In [6]:
%%bash
head -n 2 s3://abdbucketjddc/dataset/london.csv
tail -n 2 s3://abdbucketjddc/dataset/london.csv

head: cannot open ‘s3://abdbucketjddc/dataset/london.csv’ for reading: No such file or directory
tail: cannot open ‘s3://abdbucketjddc/dataset/london.csv’ for reading: No such file or directory


CalledProcessError: Command 'b'head -n 2 s3://abdbucketjddc/dataset/london.csv\ntail -n 2 s3://abdbucketjddc/dataset/london.csv\n'' returned non-zero exit status 1.

In [ ]:
! head -n 2 csv/london_stations.csv
! tail -n 2 csv/london_stations.csv

station_id,station_name,longitude,latitude
1,"River Street, Clerkenwell",-0.109971,51.5292
838,"Fore Street Avenue, Guildhall",-0.09140169999999999,51.518093
839,"Sea Containers, South Bank",-0.1068403,51.507974


In [ ]:
! head -n 2 csv/london_weather.csv
! tail -n 2 csv/london_weather.csv

date,cloud_cover,sunshine,global_radiation,max_temp,mean_temp,min_temp,precipitation,pressure,snow_depth
19790101,2.0,7.0,52.0,2.3,-4.1,-7.5,0.4,101900.0,9.0
20201230,6.0,0.4,22.0,5.6,2.7,-0.1,0.0,100200.0,
20201231,7.0,1.3,34.0,1.5,-0.8,-3.1,0.0,100500.0,


In [7]:
# Some Spark related imports we will use hereafter

import sys

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Lets read our files:

In [8]:
filename = 's3://abdbucketjddc/dataset/london.csv'
london = spark.read.csv(filename, header="true", inferSchema="true", sep=',')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
filename = 's3://abdbucketjddc/dataset/london_stations.csv'
stations = spark.read.csv(filename, header="true", inferSchema="true", sep=',')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
filename = 's3://abdbucketjddc/dataset/london_weather.csv'
weather = spark.read.csv(filename, header="true", inferSchema="true", sep=',')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Lets analyze the structure of our datasets:

#### Rentals:

In [11]:
print(london.printSchema())
london_count = london.count()
london_count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- rental_id: integer (nullable = true)
 |-- duration: double (nullable = true)
 |-- bike_id: double (nullable = true)
 |-- end_rental_date_time: timestamp (nullable = true)
 |-- end_station_id: double (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- start_rental_date_time: timestamp (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)

None
38215560

In [12]:
london.show(10, vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0--------------------------------------
 rental_id              | 61343322             
 duration               | 60.0                 
 bike_id                | 12871.0              
 end_rental_date_time   | 2016-12-28 00:01:00  
 end_station_id         | 660.0                
 end_station_name       | West Kensington S... 
 start_rental_date_time | 2016-12-28 00:00:00  
 start_station_id       | 633                  
 start_station_name     | Vereker Road Nort... 
-RECORD 1--------------------------------------
 rental_id              | 61343321             
 duration               | 300.0                
 bike_id                | 2837.0               
 end_rental_date_time   | 2016-12-28 00:05:00  
 end_station_id         | 763.0                
 end_station_name       | Mile End Park Lei... 
 start_rental_date_time | 2016-12-28 00:00:00  
 start_station_id       | 531                  
 start_station_name     | Twig Folly Bridge... 
-RECORD 2-------------------------------

#### Stations:

In [13]:
print(stations.printSchema())
stations_count = stations.count()
stations_count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- station_id: integer (nullable = true)
 |-- station_name: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)

None
802

In [14]:
stations.show(10, vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0----------------------------
 station_id   | 1                    
 station_name | River Street, Cle... 
 longitude    | -0.109971            
 latitude     | 51.5292              
-RECORD 1----------------------------
 station_id   | 2                    
 station_name | Phillimore Garden... 
 longitude    | -0.197574            
 latitude     | 51.4996              
-RECORD 2----------------------------
 station_id   | 3                    
 station_name | Christopher Stree... 
 longitude    | -0.0846057           
 latitude     | 51.5213              
-RECORD 3----------------------------
 station_id   | 4                    
 station_name | St. Chad's Street... 
 longitude    | -0.120974            
 latitude     | 51.5301              
-RECORD 4----------------------------
 station_id   | 5                    
 station_name | Sedding Street, S... 
 longitude    | -0.156876            
 latitude     | 51.4931              
-RECORD 5----------------------------
 station_id 

#### Weather

In [15]:
print(weather.printSchema())
weather_count = weather.count()
weather_count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date: integer (nullable = true)
 |-- cloud_cover: double (nullable = true)
 |-- sunshine: double (nullable = true)
 |-- global_radiation: double (nullable = true)
 |-- max_temp: double (nullable = true)
 |-- mean_temp: double (nullable = true)
 |-- min_temp: double (nullable = true)
 |-- precipitation: double (nullable = true)
 |-- pressure: double (nullable = true)
 |-- snow_depth: double (nullable = true)

None
15341

In [16]:
weather.show(10, vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0--------------------
 date             | 19790101 
 cloud_cover      | 2.0      
 sunshine         | 7.0      
 global_radiation | 52.0     
 max_temp         | 2.3      
 mean_temp        | -4.1     
 min_temp         | -7.5     
 precipitation    | 0.4      
 pressure         | 101900.0 
 snow_depth       | 9.0      
-RECORD 1--------------------
 date             | 19790102 
 cloud_cover      | 6.0      
 sunshine         | 1.7      
 global_radiation | 27.0     
 max_temp         | 1.6      
 mean_temp        | -2.6     
 min_temp         | -7.5     
 precipitation    | 0.0      
 pressure         | 102530.0 
 snow_depth       | 8.0      
-RECORD 2--------------------
 date             | 19790103 
 cloud_cover      | 5.0      
 sunshine         | 0.0      
 global_radiation | 13.0     
 max_temp         | 1.3      
 mean_temp        | -2.8     
 min_temp         | -7.2     
 precipitation    | 0.0      
 pressure         | 102050.0 
 snow_depth       | 4.0      
-RECORD 3-

### NULL Values

In [17]:
print('\nNulls in Rentals:')
cols_to_forget = []
rentals_cols_interest = [x for x in london.columns if x not in cols_to_forget]
for cl in rentals_cols_interest:
    k = london.select(cl).filter(F.col(cl).isNull()).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {london_count} records ({k*100/london_count:.2f}%)')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Nulls in Rentals:
Column duration with 67617 nulls or NaN, out of 38215560 records (0.18%)
Column bike_id with 17 nulls or NaN, out of 38215560 records (0.00%)
Column end_rental_date_time with 67617 nulls or NaN, out of 38215560 records (0.18%)
Column end_station_id with 68213 nulls or NaN, out of 38215560 records (0.18%)
Column end_station_name with 68213 nulls or NaN, out of 38215560 records (0.18%)

In [18]:
print('\nNulls in Stations:')
cols_to_forget = []
stations_cols_interest = [x for x in stations.columns if x not in cols_to_forget]
for cl in stations_cols_interest:
    k = stations.select(cl).filter(F.col(cl).isNull()).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {stations_count} records ({k*100/stations_count:.2f}%)')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Nulls in Stations:

In [19]:
print('\nNulls in Weather:')
weather_count = weather.count()
cols_to_forget = []
weather_cols_interest = [x for x in weather.columns if x not in cols_to_forget]
for cl in weather_cols_interest:
    k = weather.select(cl).filter(F.col(cl).isNull()).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {weather_count} records ({k*100/weather_count:.2f}%)')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Nulls in Weather:
Column cloud_cover with 19 nulls or NaN, out of 15341 records (0.12%)
Column global_radiation with 19 nulls or NaN, out of 15341 records (0.12%)
Column max_temp with 6 nulls or NaN, out of 15341 records (0.04%)
Column mean_temp with 36 nulls or NaN, out of 15341 records (0.23%)
Column min_temp with 2 nulls or NaN, out of 15341 records (0.01%)
Column precipitation with 6 nulls or NaN, out of 15341 records (0.04%)
Column pressure with 4 nulls or NaN, out of 15341 records (0.03%)
Column snow_depth with 1441 nulls or NaN, out of 15341 records (9.39%)

'snow_depth' better to remove. 
Let´s analyze if we can fill the other features with null values

#### Check for outliers

In [20]:
cols_interest = []
print('\nFeature describe for Rentals:')
london.describe().show(vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Feature describe for Rentals:
-RECORD 0----------------------------------
 summary            | count                
 rental_id          | 38215560             
 duration           | 38147943             
 bike_id            | 38215543             
 end_station_id     | 38147347             
 end_station_name   | 38147347             
 start_station_id   | 38215560             
 start_station_name | 38215560             
-RECORD 1----------------------------------
 summary            | mean                 
 rental_id          | 8.138741601914042E7  
 duration           | 1286.4046914665885   
 bike_id            | 8741.978706412729    
 end_station_id     | 373.7964623594925    
 end_station_name   | null                 
 start_station_id   | 375.993760133307     
 start_station_name | null                 
-RECORD 2----------------------------------
 summary            | stddev               
 rental_id          | 1.1536705978260422E7 
 duration           | 4058.5640765654603   
 

In [21]:
cols_interest = []
print('\nFeature describe for Stations:')
stations.describe().show(vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Feature describe for Stations:
-RECORD 0----------------------------
 summary      | count                
 station_id   | 802                  
 station_name | 802                  
 longitude    | 802                  
 latitude     | 802                  
-RECORD 1----------------------------
 summary      | mean                 
 station_id   | 418.5224438902743    
 station_name | null                 
 longitude    | -0.12803249000997505 
 latitude     | 51.5057102121322     
-RECORD 2----------------------------
 summary      | stddev               
 station_id   | 243.94179248820723   
 station_name | null                 
 longitude    | 0.05524560719993001  
 latitude     | 0.021132255816371483 
-RECORD 3----------------------------
 summary      | min                  
 station_id   | 1                    
 station_name | Abbey Orchard Str... 
 longitude    | -0.23676999999999998 
 latitude     | 51.3341              
-RECORD 4----------------------------
 summary      | ma

A duration of 1984694 seconds, seems like an outlier

In [22]:
print('\nFeature describe for Weather:')
weather.describe().show(vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Feature describe for Weather:
-RECORD 0-------------------------------
 summary          | count               
 date             | 15341               
 cloud_cover      | 15322               
 sunshine         | 15341               
 global_radiation | 15322               
 max_temp         | 15335               
 mean_temp        | 15305               
 min_temp         | 15339               
 precipitation    | 15335               
 pressure         | 15337               
 snow_depth       | 13900               
-RECORD 1-------------------------------
 summary          | mean                
 date             | 1.999567159344241E7 
 cloud_cover      | 5.268241743897663   
 sunshine         | 4.350237924515966   
 global_radiation | 118.75695078971414  
 max_temp         | 15.388777306814523  
 mean_temp        | 11.475511270826521  
 min_temp         | 7.559867005671808   
 precipitation    | 1.6686338441474187  
 pressure         | 101536.6055943144   
 snow_depth       | 0.0379

#### Unique values

In [23]:
columns = [x for x in london.columns]
london_count = london.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
print('\nUnique values (Rentals):')
for cl in columns:
    k = london.select(cl).distinct().count()
    print(f'Column {cl} with {k} distinct values, out of {london_count} records ({k*100/london_count:.2f}%)')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Unique values (Rentals):
Column rental_id with 38215560 distinct values, out of 38215560 records (100.00%)
Column duration with 20597 distinct values, out of 38215560 records (0.05%)
Column bike_id with 16020 distinct values, out of 38215560 records (0.04%)
Column end_rental_date_time with 1779017 distinct values, out of 38215560 records (4.66%)
Column end_station_id with 809 distinct values, out of 38215560 records (0.00%)
Column end_station_name with 841 distinct values, out of 38215560 records (0.00%)
Column start_rental_date_time with 1780309 distinct values, out of 38215560 records (4.66%)
Column start_station_id with 807 distinct values, out of 38215560 records (0.00%)
Column start_station_name with 839 distinct values, out of 38215560 records (0.00%)

From the results above two conclusions can be taken:
- There are 16019 Bikes in total
- There are inconsitency between the Station number, since the start_station_id must not be different from the end_station_id. Thus the 2 extra station will be dropped from the dataframe.

In [25]:
columns = [x for x in stations.columns]
stations_count = stations.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
print('\nUnique values (Stations):')
for cl in columns:
    k = stations.select(cl).distinct().count()
    print(f'Column {cl} with {k} distinct values, out of {stations_count} records ({k*100/stations_count:.2f}%)')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Unique values (Stations):
Column station_id with 802 distinct values, out of 802 records (100.00%)
Column station_name with 802 distinct values, out of 802 records (100.00%)
Column longitude with 792 distinct values, out of 802 records (98.75%)
Column latitude with 505 distinct values, out of 802 records (62.97%)

In [27]:
columns = [x for x in weather.columns]
weather_count = weather.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
print('\nUnique values (Weather):')
for cl in columns:
    k = weather.select(cl).distinct().count()
    print(f'Column {cl} with {k} distinct values, out of {weather_count} records ({k*100/weather_count:.2f}%)')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Unique values (Weather):
Column date with 15341 distinct values, out of 15341 records (100.00%)
Column cloud_cover with 11 distinct values, out of 15341 records (0.07%)
Column sunshine with 160 distinct values, out of 15341 records (1.04%)
Column global_radiation with 361 distinct values, out of 15341 records (2.35%)
Column max_temp with 375 distinct values, out of 15341 records (2.44%)
Column mean_temp with 322 distinct values, out of 15341 records (2.10%)
Column min_temp with 297 distinct values, out of 15341 records (1.94%)
Column precipitation with 263 distinct values, out of 15341 records (1.71%)
Column pressure with 643 distinct values, out of 15341 records (4.19%)
Column snow_depth with 20 distinct values, out of 15341 records (0.13%)

#### Lets check for duplicates:

In [29]:
print('\nHow many duplicates (Rentals):')

print(london.count() - london.dropDuplicates().count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


How many duplicates (Rentals):
0

In [30]:
print('\nHow many duplicates (Stations):')

print(stations.count() - stations.dropDuplicates().count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


How many duplicates (Stations):
0

In [31]:
print('\nHow many duplicates (Weather):')

print(weather.count() - weather.dropDuplicates().count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


How many duplicates (Weather):
0

#### Other tests

##### Checking if every end date is after every start date (Rentals):

In [32]:
print('\nIncorrect in dates:')
print(london.filter(F.col('end_rental_date_time') <= F.col('start_rental_date_time')).count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Incorrect in dates:
8220

##### Check if we have any rental with a duration lesser or equal to 0 (Rentals):

In [33]:
print(london.select('duration').filter(F.col('duration') <= 0).count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5582

## Actions

### Rentals

#### Filter for every valid date period

In [34]:
london = london.filter(F.col('end_rental_date_time') > F.col('start_rental_date_time'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Dropping rows in with null values:

In [35]:
london_nonull = london.na.drop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
london_nonull_count = london_nonull.count()
london_nonull_count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

38139058

Difference between both datasets:

In [37]:
print(london_count - london_nonull_count)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

76502

#### Adding a column for: year, month, day, hour

In [38]:
from pyspark.sql.functions import year
from pyspark.sql.functions import month
from pyspark.sql.functions import hour, dayofweek
from pyspark.sql.functions import to_date, date_format

new_london = ( london_nonull
              .withColumn('Year', year(F.col('start_rental_date_time')))
              .withColumn('Month', month(F.col('start_rental_date_time')))
              .withColumn("Day", date_format(F.col('start_rental_date_time'), "d"))
              .withColumn('Hour', hour(F.col('start_rental_date_time')))
              .withColumn('DayofWeek', dayofweek(F.col('start_rental_date_time')))
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
new_london.show(5, vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0--------------------------------------
 rental_id              | 61343322             
 duration               | 60.0                 
 bike_id                | 12871.0              
 end_rental_date_time   | 2016-12-28 00:01:00  
 end_station_id         | 660.0                
 end_station_name       | West Kensington S... 
 start_rental_date_time | 2016-12-28 00:00:00  
 start_station_id       | 633                  
 start_station_name     | Vereker Road Nort... 
 Year                   | 2016                 
 Month                  | 12                   
 Day                    | 28                   
 Hour                   | 0                    
 DayofWeek              | 4                    
-RECORD 1--------------------------------------
 rental_id              | 61343321             
 duration               | 300.0                
 bike_id                | 2837.0               
 end_rental_date_time   | 2016-12-28 00:05:00  
 end_station_id         | 763.0         

#import the dataframe

filename = 'new_csv.csv'

new_london = spark.read.csv(filename, header="true", inferSchema="true", sep=',')

### Weather

#### Fill 'null' values with the backwards fill method.

In [40]:
from pyspark.sql.functions import mean, last, col
from pyspark.sql.window import Window
# Assume `df` is your DataFrame

# Compute the mean of the column with null values
#mean_value = new_weather.select(mean(col("mean"))).collect()[0][0]

# Replace null values with the mean
#new_weather = new_weather.fillna(mean_value, subset=["mean"])

#bfill

w = Window.orderBy("date").rowsBetween(Window.unboundedPreceding, -1)

weather = (weather
                .withColumn("mean_temp_test", last(col("mean_temp"), True).over(w))
                .withColumn('cloud_cover_test',last(col('cloud_cover'), True).over(w))
                .withColumn('global_radiation_test',last(col('global_radiation'), True).over(w))
                .withColumn('max_temp_test',last(col('max_temp'), True).over(w))
                .withColumn('min_temp_test',last(col('min_temp'), True).over(w))
                .withColumn('precipitation_test',last(col('precipitation'), True).over(w))
                .withColumn('pressure_test',last(col('pressure'), True).over(w))
          )

#weather.filter(F.col('mean_test').isNull()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
from pyspark.sql.functions import when, col

new_mean = when(col('mean_temp').isNull(), col('mean_temp_test')).otherwise(col('mean_temp'))
new_cloud = when(col('cloud_cover').isNull(), col('cloud_cover_test')).otherwise(col('cloud_cover'))
new_gr = when(col('global_radiation').isNull(), col('global_radiation_test')).otherwise(col('global_radiation'))
new_maxt = when(col('max_temp').isNull(), col('max_temp_test')).otherwise(col('max_temp'))
new_mint = when(col('min_temp').isNull(), col('min_temp_test')).otherwise(col('min_temp'))
new_prec = when(col('precipitation').isNull(), col('precipitation_test')).otherwise(col('precipitation'))
new_press = when(col('pressure').isNull(), col('pressure_test')).otherwise(col('pressure'))

new_weather = (weather
                .withColumn('mean', new_mean)
                .withColumn('cloud', new_cloud)
                .withColumn('gr', new_gr)
                .withColumn('maxt', new_maxt)
                .withColumn('mint', new_mint)
                .withColumn('prec', new_prec)
                .withColumn('press', new_press)
              )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Dropping 'snow_depth' feature

In [42]:
new_weather = new_weather.drop(F.col('snow_depth'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Check if changes were efective

In [43]:
print('\nNulls in Weather:')
new_weather_count = new_weather.count()
cols_to_forget = ['cloud_cover', 'global_radiation', 'max_temp', 'mean_temp', 'min_temp', 'precipitation', 'pressure',"mean_temp_test",'cloud_cover_test','global_radiation_test', 'max_temp_test', 'min_temp_test', 'precipitation_test', 'pressure_test']
weather_cols_interest = [x for x in new_weather.columns if x not in cols_to_forget]
for cl in weather_cols_interest:
    k = new_weather.select(cl).filter(F.col(cl).isNull()).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {new_weather_count} records ({k*100/new_weather_count:.2f}%)')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Nulls in Weather:

In [44]:
cols_to_forget = ['cloud_cover', 'global_radiation', 'max_temp', 'mean_temp', 'min_temp', 'precipitation', 'pressure',"mean_temp_test",'cloud_cover_test','global_radiation_test', 'max_temp_test', 'min_temp_test', 'precipitation_test', 'pressure_test']

weather_cols_interest = [x for x in new_weather.columns if x not in cols_to_forget]

new_weather.select(weather_cols_interest).describe().show(vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0-----------------------
 summary  | count               
 date     | 15341               
 sunshine | 15341               
 mean     | 15341               
 cloud    | 15341               
 gr       | 15341               
 maxt     | 15341               
 mint     | 15341               
 prec     | 15341               
 press    | 15341               
-RECORD 1-----------------------
 summary  | mean                
 date     | 1.999567159344241E7 
 sunshine | 4.350237924515966   
 mean     | 11.472876605175669  
 cloud    | 5.26777915390131    
 gr       | 118.89172804901897  
 maxt     | 15.389700801773074  
 mint     | 7.560093866110414   
 prec     | 1.6684635942898551  
 press    | 101536.65927905613  
-RECORD 2-----------------------
 summary  | stddev              
 date     | 121217.55827681546  
 sunshine | 4.028339404679589   
 mean     | 5.728950610266871   
 cloud    | 2.0702455817982637  
 gr       | 88.95028904838944   
 maxt     | 6.555536453716224   
 mint     

### Concatenate all datasets:

In [45]:
from pyspark.sql.functions import date_format

new_london = new_london.withColumn("dateconc",  date_format(F.col("start_rental_date_time"), "yyyyMMdd"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
new_london.show(1, vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0--------------------------------------
 rental_id              | 61343322             
 duration               | 60.0                 
 bike_id                | 12871.0              
 end_rental_date_time   | 2016-12-28 00:01:00  
 end_station_id         | 660.0                
 end_station_name       | West Kensington S... 
 start_rental_date_time | 2016-12-28 00:00:00  
 start_station_id       | 633                  
 start_station_name     | Vereker Road Nort... 
 Year                   | 2016                 
 Month                  | 12                   
 Day                    | 28                   
 Hour                   | 0                    
 DayofWeek              | 4                    
 dateconc               | 20161228             
only showing top 1 row

In [47]:
from pyspark.sql.functions import sum,avg,max,count

group_cols = ["start_station_id", "Hour","Day",'DayofWeek', "Month", "Year","dateconc","end_station_id","end_station_name"]
grouped = new_london.groupBy(group_cols) \
            .agg(sum("duration").alias("sum_duration"), \
                 avg("duration").alias("avg_duration"), \
                 count('duration').alias('num_rentals')
                 )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
grouped = grouped.join(stations, grouped.start_station_id == stations.station_id, "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
columns_to_drop = ['end_station_name']
grouped = grouped.select([column for column in grouped.columns if column not in columns_to_drop])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
from pyspark.sql.functions import substring_index

#create a new column called "start_area" with values after the comma (area name)

# Define the delimiter (',') and the index (1) after which to extract the values
delimiter = ","
index = -1

# Create the new column using substring_index
grouped = grouped.withColumn("area", substring_index(grouped.station_name, delimiter, index))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
from pyspark.sql.functions import col, regexp_replace
grouped = grouped.withColumn("area_cleaned", regexp_replace(col("area"), r"['.\s]+", ""))

#unique_values = unique_values.select('area_test')
unique_values = grouped.select("area_cleaned","area").orderBy("area_cleaned","area").distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
cols_to_forget = ['cloud_cover', 'global_radiation', 'max_temp', 'mean_temp', 'min_temp', 'precipitation', 'pressure',"mean_temp_test",'cloud_cover_test','global_radiation_test', 'max_temp_test', 'min_temp_test', 'precipitation_test', 'pressure_test']

weather_cols_interest = [x for x in new_weather.columns if x not in cols_to_forget]
new_weather = new_weather.select(weather_cols_interest)
grouped = grouped.join(new_weather,grouped.dateconc == new_weather.date, "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
grouped = grouped.drop(F.col('area'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
bikes_and_stations = grouped

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Check concatenated dataset data quality

In [55]:
print(bikes_and_stations.printSchema())
bikes_and_stations_count = london.count()
bikes_and_stations_count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_station_id: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Day: string (nullable = true)
 |-- DayofWeek: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- dateconc: string (nullable = true)
 |-- end_station_id: double (nullable = true)
 |-- sum_duration: double (nullable = true)
 |-- avg_duration: double (nullable = true)
 |-- num_rentals: long (nullable = false)
 |-- station_id: integer (nullable = true)
 |-- station_name: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- area_cleaned: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- sunshine: double (nullable = true)
 |-- mean: double (nullable = true)
 |-- cloud: double (nullable = true)
 |-- gr: double (nullable = true)
 |-- maxt: double (nullable = true)
 |-- mint: double (nullable = true)
 |-- prec: double (nullable = true)
 |-- press: double (nullable = true)


In [56]:
bikes_and_stations.show(10, vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0--------------------------------
 start_station_id | 299                  
 Hour             | 13                   
 Day              | 16                   
 DayofWeek        | 5                    
 Month            | 2                    
 Year             | 2017                 
 dateconc         | 20170216             
 end_station_id   | 424.0                
 sum_duration     | 360.0                
 avg_duration     | 360.0                
 num_rentals      | 1                    
 station_id       | 299                  
 station_name     | Vincent Square, W... 
 longitude        | -0.136929            
 latitude         | 51.494               
 area_cleaned     | Westminster          
 date             | 20170216             
 sunshine         | 2.5                  
 mean             | 8.6                  
 cloud            | 6.0                  
 gr               | 57.0                 
 maxt             | 13.5                 
 mint             | 4.8           

In [57]:
print('\nNulls in dataframe:')
cols_to_forget = []
bikes_and_stations_cols_int = [x for x in bikes_and_stations.columns if x not in cols_to_forget]
for cl in bikes_and_stations_cols_int:
    k = bikes_and_stations.select(cl).filter(F.col(cl).isNull()).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {bikes_and_stations_count} records ({k*100/bikes_and_stations_count:.2f}%)')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Nulls in dataframe:
Column station_id with 3780 nulls or NaN, out of 38139723 records (0.01%)
Column station_name with 3780 nulls or NaN, out of 38139723 records (0.01%)
Column longitude with 3780 nulls or NaN, out of 38139723 records (0.01%)
Column latitude with 3780 nulls or NaN, out of 38139723 records (0.01%)
Column area_cleaned with 3780 nulls or NaN, out of 38139723 records (0.01%)

In [58]:
cols_interest = []
print('\nFeature describe for dataframe:')
bikes_and_stations.describe().show(vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Feature describe for dataframe:
-RECORD 0--------------------------------
 summary          | count                
 start_station_id | 32230756             
 Hour             | 32230756             
 Day              | 32230756             
 DayofWeek        | 32230756             
 Month            | 32230756             
 Year             | 32230756             
 dateconc         | 32230756             
 end_station_id   | 32230756             
 sum_duration     | 32230756             
 avg_duration     | 32230756             
 num_rentals      | 32230756             
 station_id       | 32226976             
 station_name     | 32226976             
 longitude        | 32226976             
 latitude         | 32226976             
 area_cleaned     | 32226976             
 date             | 32230756             
 sunshine         | 32230756             
 mean             | 32230756             
 cloud            | 32230756             
 gr               | 32230756             
 

In [59]:
print('\nHow many duplicates:')

print(bikes_and_stations.count() - bikes_and_stations.dropDuplicates().count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


How many duplicates:
0

In [64]:
print('\nUnique values:')
for cl in bikes_and_stations.columns:
    k = bikes_and_stations.select(cl).distinct().count()
    print(f'Column {cl} with {k} distinct values, out of {bikes_and_stations_count} records ({k*100/bikes_and_stations_count:.2f}%)')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interrupted by user


### Actions

In [61]:
bikes_and_stations = bikes_and_stations.na.drop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
bikes_and_stations.write.mode("overwrite").parquet("s3://abdbucketjddc/dataout/londonbikes")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Visualizing data

In [ ]:
def plotBar(df, xcol, ycol, huecol=None):
    return sns.barplot(data=df, x=xcol, y=ycol, hue=huecol)

In [ ]:
def plotCorrelationMatrix(df, annot=False):
    # compute the correlation matrix
    corr = df.corr()
    
    # generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=bool))

    # set up the matplotlib figure
    f, ax = plt.subplots(figsize=(20, 9))

    # generate a custom colormap
    #cmap = sns.divergent_palette(230, 20, as_cmap=True)
    
    cmap='coolwarm'

    # draw the heatmap with the mask and correct aspect ratio
    return sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, annot=annot,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
df_plot = ( bikes_and_stations
                .groupby('Year')
                .count()
                .sort('Year', ascending=True)
                .toPandas()
          )
plotBar(df_plot, 'Year', 'count')
plt.title('Number of rentals in London by year')
plt.show()

In [ ]:
df_plot = ( bikes_and_stations
                .groupby('Month')
                .count()
                .sort('Month', ascending=True)
                .toPandas()
          )
plotBar(df_plot, 'Month', 'count')
plt.title('Number of rentals in London by month')
plt.show()

In [ ]:
df_plot = ( bikes_and_stations
                .groupby('Hour')
                .count()
                .sort('Hour', ascending=True)
                .toPandas()
          )
plotBar(df_plot, 'Hour', 'count')
plt.title('Number of rentals in London by Hour')
plt.show()

In [ ]:
df_plot = ( bikes_and_stations
                .groupby('DayofWeek')
                .count()
                .sort('DayofWeek', ascending=True)
                .toPandas()
          )
plotBar(df_plot, 'DayofWeek', 'count')
plt.title('Number of rentals in London by day of the week')
plt.show()

In [ ]:
df_plot = ( bikes_and_stations
                .groupby('mean')
                .count()
                .sort('mean', ascending=True)
                .toPandas()
          )
plotBar(df_plot, 'mean', 'count')
plt.title('Number of rentals in London by mean temperature')
plt.show()

In [ ]:
df_plot = ( bikes_and_stations
                .groupby('sunshine')
                .count()
                .sort('sunshine', ascending=True)
                .toPandas()
          )
plotBar(df_plot, 'sunshine', 'count')
plt.title('Number of rentals in London by sunshine')
plt.show()

In [ ]:
df_plot = ( bikes_and_stations
                .groupby('press')
                .count()
                .sort('press', ascending=True)
                .toPandas()
          )
plotBar(df_plot, 'press', 'count')
plt.title('Number of rentals in London by pressure')
plt.show()

In [ ]:
df_plot = ( bikes_and_stations
                .groupby('prec')
                .count()
                .sort('prec', ascending=True)
                .toPandas()
          )
plotBar(df_plot, 'prec', 'count')
plt.title('Number of rentals in London by precipitation')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a PySpark DataFrame called 'bikes_and_stations' and a column named 'start_area'
df_plot = grouped.groupBy('area_cleaned').count().orderBy('count', ascending=True).limit(15).toPandas()

# Creating the horizontal bar plot
plt.barh(df_plot['area_cleaned'], df_plot['count'])

# Adding labels and title
plt.xlabel('Count')
plt.ylabel('Area')
plt.title('Area with most rentals')

# Displaying the plot
plt.show()


### Map Visualizing:

In [ ]:
from pyspark.sql.functions import sum,avg,max,count

group_cols = ["start_station_id", "dateconc"]

grouped = joined.groupBy(group_cols) \
            .agg(sum("duration").alias("sum_duration"), \
                 avg("duration").alias("avg_duration"), \
                 count('duration').alias('num_rentals')
                 )

grouped.show(1, truncate=False, vertical=True)

In [ ]:
joined = grouped.join(new_weather, grouped.dateconc == new_weather.date, 'left')

joined.show(5, vertical=True)

In [ ]:
#!pip install folium
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

df = joined.toPandas()

# find the row of the house which has the highest price
maxpr=df.loc[df['num_rentals'].idxmax()]

# define a function to draw a basemap easily
def generateBaseMap(default_location=[51.5292, -0.109971], default_zoom_start=9.4):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

df_copy = df.copy()
df_copy['count'] = 1
basemap = generateBaseMap()
# add carton position map
folium.TileLayer('cartodbpositron').add_to(basemap)
s=folium.FeatureGroup(name='icon').add_to(basemap)
# add a marker for the house which has the highest price
folium.Marker([maxpr['latitude'], maxpr['longitude']],popup='Highest Rentals: $'+str(format(maxpr['num_rentals'],'.0f')),
              icon=folium.Icon(color='green')).add_to(s)
# add heatmap
HeatMap(data=df_copy[['latitude','longitude','count']].groupby(['latitude','longitude']).sum().reset_index().values.tolist(),
        radius=8,max_zoom=13,name='Heat Map').add_to(basemap)
folium.LayerControl(collapsed=False).add_to(basemap)
basemap